In [1]:
import os
import cv2
import time
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

lst = {0:'w', 1:'a', 2:'s', 3:'d', 4:'nop'}
REBUILD_DATA = False # set to true to one once, then back to false unless you want to change something in your training data.

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
class prep():
    IMG_SIZE = 50
    
    training_data = []
    address = []
    LABELS = {}
    count = [0,0,0,0,0]
    
    for idx in range(5):
        address.append('data/{}'.format(lst[idx]))
        LABELS[address[idx]] = idx

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(5)[self.LABELS[label]]]) 

                        self.count[self.LABELS[label]] += 1

                    except Exception as e:
                        print(label, f, str(e))
                        pass

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        
        print(self.count)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 5) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)

In [5]:
def train(net):
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_function = nn.MSELoss()
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            net.zero_grad()

            optimizer.zero_grad()   # zero the gradient buffers
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")


def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]  # returns a list, 
            predicted_class = torch.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

In [8]:
REBUILD_DATA = False

net = Net().to(device)
print(net)

if REBUILD_DATA:
    tommy = prep()
    tommy.make_training_data()

training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

BATCH_SIZE = 100
EPOCHS = 20

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=5, bias=True)
)
4351


In [9]:
train(net)

 32%|██████████████████████████▎                                                      | 13/40 [00:00<00:00, 126.18it/s]

Epoch: 0. Loss: 0.14967411756515503


 32%|██████████████████████████▎                                                      | 13/40 [00:00<00:00, 127.41it/s]

Epoch: 1. Loss: 0.14047186076641083


 32%|██████████████████████████▎                                                      | 13/40 [00:00<00:00, 126.79it/s]

...

Epoch: 17. Loss: 0.00018642758368514478


 32%|██████████████████████████▎                                                      | 13/40 [00:00<00:00, 124.99it/s]

Epoch: 18. Loss: 0.0001254161907127127


100%|█████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 123.28it/s]

Epoch: 19. Loss: 6.556265725521371e-05


In [10]:
test_X.to(device)
test_y.to(device)

test(net)

100%|███████████████████████████████████████████████████████████████████████████████| 435/435 [00:00<00:00, 975.34it/s]

Accuracy:  0.938


In [12]:
t = time.strftime("%b%d_%H%M_", time.localtime())
PATH = t + "model.pt"
print(PATH)

torch.save(net.state_dict(), PATH)

Jul30_0940_model.pt
